In [5]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sb
from xgboost import XGBClassifier,XGBRegressor
import gc
import warnings
warnings.filterwarnings("ignore")

## Parameters

In [6]:
RAND_VAL=42
num_folds=5

## Read Source Files

In [7]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("ObesityDataSet.csv")
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [10]:
df_train_orig=pd.read_csv("ObesityDataSet.csv")
df_train_orig['id']=-9999
df_train_orig=df_train_orig[df_train.columns]
df_train_orig.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,-9999,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,-9999,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,-9999,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,-9999,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,-9999,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [11]:
x=df_train.iloc[:1,:]
x.values

array([[0, 'Male', 24.443011, 1.699998, 81.66995, 'yes', 'yes', 2.0,
        2.983297, 'Sometimes', 'no', 2.763573, 'no', 0.0, 0.976473,
        'Sometimes', 'Public_Transportation', 'Overweight_Level_II']],
      dtype=object)

### Merge Files

In [12]:
df_train['typ']=0
df_train_orig['typ']=0
df_test['typ']=1
#
df_all=pd.concat([df_train,df_train_orig,df_test],axis=0)

## Feature Engineering

In [13]:
cat_feats = df_all.columns[np.where(df_all.dtypes == "object")[0]].drop('NObeyesdad')
num_feats  = df_all.columns[np.where(df_all.dtypes != "object")[0]].drop(['id','typ'])
##
df_all=pd.get_dummies(df_all,columns=cat_feats)
sc=StandardScaler()
df_all[num_feats]=sc.fit_transform(df_all[num_feats])

In [14]:
df_train = df_all[df_all.typ==0].reset_index(drop=True)
df_test = df_all[df_all.typ==1].reset_index(drop=True)
##
print(len(df_train))
print(len(df_test))
##
feat_cols=df_train.columns.drop(['id','typ','NObeyesdad'])
X_test=df_test[feat_cols]
##
print(feat_cols)
df_train.head()

22869
2111
Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes',
       'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no',
       'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'CALC_Always',
       'CALC_Frequently', 'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile',
       'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation',
       'MTRANS_Walking'],
      dtype='object')


,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,NObeyesdad,...,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0.0,0.089832,-0.005535,-0.227615,-0.827460,0.326652,1.210965,-1.173919,0.584605,Overweight_Level_II,...,False,False,False,True,False,False,False,False,True,False
1,1.0,-1.019732,-1.590132,-1.163833,-0.827460,0.349893,-0.042348,0.015978,0.623594,Normal_Weight,...,False,False,False,False,True,True,False,False,False,False
2,2.0,-1.019732,0.124200,-1.423190,-1.051430,-1.860136,-0.189452,-0.143415,1.739851,Insufficient_Weight,...,False,False,False,False,True,False,False,False,True,False
3,3.0,-0.511235,0.115937,1.654877,1.047303,0.349893,-0.577337,0.572687,0.259342,Obesity_Type_III,...,False,False,False,True,False,False,False,False,True,False
4,4.0,1.329426,2.418796,0.232643,0.446749,-1.081232,-0.075425,1.167766,0.510442,Overweight_Level_II,...,False,False,False,True,False,False,False,False,True,False


In [15]:
X=df_train[feat_cols]
y=df_train['NObeyesdad']
#
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X.iloc[:1,].values

array([[0.08983179098898399, -0.005535251315383527, -0.22761513805582662,
        -0.8274596658812211, 0.32665195182795076, 1.2109652971831004,
        -1.1739193444072347, 0.5846051216350999, False, True, False,
        True, False, True, False, False, True, False, True, False, True,
        False, False, False, True, False, False, False, False, True,
        False]], dtype=object)

In [16]:
xgb_params ={
    'learning_rate': 0.026,
    'n_estimators': 1500,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'early_stopping_rounds':60,
    'max_depth': 19,
    'min_child_weight': 11.7,
    'gamma': 0.40,
    'colsample_bytree': 0.46,
    'random_state':RAND_VAL,
    'subsample': 0.82,
    'reg_alpha': 0.0004,
    'verbosity': 0,
    'reg_lambda': 1.4,
}

## Training

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


XGB=XGBClassifier(**xgb_params)
XGB.fit(X_train,y_train, eval_set=[(X_val, y_val)],verbose=200)

[0]	validation_0-mlogloss:1.89248
[200]	validation_0-mlogloss:0.31469
[400]	validation_0-mlogloss:0.25783
[600]	validation_0-mlogloss:0.25153
[731]	validation_0-mlogloss:0.25129


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.46, device=None, early_stopping_rounds=60,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=0.4, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.026, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=19,
              max_leaves=None, min_child_weight=11.7, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1500,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

## Classification Report

In [18]:
y_pred_val = XGB.predict(X_val[feat_cols])
y_pred_val = y_pred_val.astype('int')
acc_val = accuracy_score(y_val, y_pred_val)
print("Accuracy : ",acc_val)

Accuracy :  0.9144439586066171


## Confusion Matrix

In [19]:
print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       838
           1       0.89      0.90      0.89      1011
           2       0.92      0.89      0.90       978
           3       0.97      0.97      0.97      1064
           4       1.00      1.00      1.00      1311
           5       0.83      0.78      0.80       815
           6       0.82      0.85      0.83       844

    accuracy                           0.91      6861
   macro avg       0.91      0.91      0.91      6861
weighted avg       0.91      0.91      0.91      6861



## Submission

In [20]:
feat_cols

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes',
       'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no',
       'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'CALC_Always',
       'CALC_Frequently', 'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile',
       'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation',
       'MTRANS_Walking'],
      dtype='object')